In [2]:
#!pip install pandas
import pandas as pd

# Analyze & Preprocess

In [3]:
# Load the CSV
df = pd.read_csv("tsa_claims.csv")
df.describe()


C:\Users\vikra\AppData\Local\Temp\ipykernel_10216\248160499.py:2: DtypeWarning: Columns (0,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("tsa_claims.csv")


,Claim Number,Date Received,Incident Date,Airport Code,Airport Name,Airline Name,Claim Type,Claim Site,Item,Claim Amount,Status,Close Amount,Disposition
count,204267,204004,202084,195743,195743,169893,196354,203527,200301,200224,204262,135315,131359
unique,204258,4007,26807,465,485,318,11,6,4704,36117,15,17162,3
top,2004050450432,5-Dec-03,12/27/2004 0:00,LAX,John F. Kennedy International,American Airlines,Passenger Property Loss,Checked Baggage,Other,$0.00,Denied,$0.00,Deny
freq,2,682,144,9596,9232,17553,117868,159753,36908,28445,68339,71546,68382


In [4]:
df.head()

,Claim Number,Date Received,Incident Date,Airport Code,Airport Name,Airline Name,Claim Type,Claim Site,Item,Claim Amount,Status,Close Amount,Disposition
0,0909802M,4-Jan-02,12/12/2002 0:00,EWR,Newark International Airport,Continental Airlines,Property Damage,Checkpoint,Other,$350.00,Approved,$350.00,Approve in Full
1,0202417M,2-Feb-02,1/16/2004 0:00,SEA,Seattle-Tacoma International,NaN,Property Damage,Checked Baggage,Luggage (all types including footlockers),$100.00,Settled,$50.00,Settle
2,0202445M,4-Feb-02,11/26/2003 0:00,STL,Lambert St. Louis International,American Airlines,Property Damage,Checked Baggage,Cell Phones,$278.88,Settled,$227.92,Settle
3,0909816M,7-Feb-02,1/6/2003 0:00,MIA,Miami International Airport,American Airlines,Property Damage,Checkpoint,Luggage (all types including footlockers),$50.00,Approved,$50.00,Approve in Full
4,2005032379513,18-Feb-02,2/5/2005 0:00,MCO,Orlando International Airport,Delta (Song),Property Damage,Checkpoint,Baby - Strollers; car seats; playpen; etc.,$84.79,Approved,$84.79,Approve in Full


### Convert datetime to dates, dollar figures (str) to floats

In [5]:
# Dates
from datetime import datetime

def parse_date_received(date_str):
    if pd.isna(date_str):
        return pd.NaT
    for fmt in ('%d-%b-%y', '%m/%d/%Y %H:%M', '%m/%d/%Y'):
        try:
            return datetime.strptime(date_str.strip(), fmt)
        except (ValueError, TypeError):
            continue
    return pd.NaT

df['Incident Date'] = pd.to_datetime(df['Incident Date'], errors='coerce', format='%m/%d/%Y', exact=False)
df['Date Received'] = df['Date Received'].apply(parse_date_received)

# Dollar amounts
df['Close Amount'] = (
    df['Close Amount']
    .astype(str)                                      # Ensure it's string type
    .str.replace('$', '', regex=False)                # Remove dollar sign
    .str.replace(';', '', regex=False)                # Remove semicolons
    .str.replace(',', '', regex=False)                # Just in case some have commas
    .str.strip()                                       # Trim spaces
    .replace('', pd.NA)                                # Replace empty strings with NaN
    .astype(float)                                     # Convert to float
)
df['Claim Amount'] = (
    df['Close Amount']
    .astype(str)                            
    .str.replace('$', '', regex=False)           
    .str.replace(';', '', regex=False)         
    .str.replace(',', '', regex=False)          
    .str.strip()                                    
    .replace('', pd.NA)                           
    .astype(float)                                    
)

### Total # Rows, Distinct Years, # Nulls

In [6]:
# Show total number of rows
print(f"Total number of rows: {len(df)}")

# Show distinct years in 'Incident Date'
distinct_years_inc = sorted(int(y) for y in df['Incident Date'].dropna().dt.year.unique())
distinct_years_rec = sorted(int(y) for y in df['Date Received'].dropna().dt.year.unique())
print(f"\nDistinct years available in 'Incident Date': {sorted(distinct_years_inc)}")
print(f"\nDistinct years available in 'Date Received': {sorted(distinct_years_rec)}")

# Before 2002
before_2002 = df[df['Incident Date'] < '2002-01-01']
print(f"Rows before 2002: {len(before_2002)}")

# After 2015
after_2015 = df[df['Date Received'] > '2015-12-31']
print(f"Rows after 2015: {len(after_2015)}")

# Rows between 2013-2015
rows_2013_2015 = df[
    (df['Date Received'] >= '2013-01-01') &
    (df['Date Received'] <= '2015-12-31')
]

print(f"Number of rows from 2013 to 2015: {len(rows_2013_2015)}")

# Count rows with no 'Incident Date'
missing_incident_date = df['Incident Date'].isnull().sum()
print(f"\nNumber of rows with no 'Incident Date': {missing_incident_date}")

# Count rows with no 'Date Received'
missing_received_date = df['Date Received'].isnull().sum()
print(f"Number of rows with no 'Date Received': {missing_received_date}")

# Show number of nulls per column
null_counts = df.isnull().sum()
print("\nNumber of null rows per column:")
print(null_counts)

Total number of rows: 204267

Distinct years available in 'Incident Date': [1996, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013]

Distinct years available in 'Date Received': [1994, 1996, 2000, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2020, 2025, 2040, 2044, 2055]
Rows before 2002: 30
Rows after 2015: 9
Number of rows from 2013 to 2015: 27220

Number of rows with no 'Incident Date': 19870
Number of rows with no 'Date Received': 263

Number of null rows per column:
Claim Number         0
Date Received      263
Incident Date    19870
Airport Code      8524
Airport Name      8524
Airline Name     34374
Claim Type        7913
Claim Site         740
Item              3966
Claim Amount     68952
Status               5
Close Amount     68952
Disposition      72908
dtype: int64


### Add 'Date Received' as 'Incident Date' if the latter doesn't exist; count # rows where both dates missing and drop

In [7]:
print("Replacing empty Incident Dates with Date Received")
df['Incident Date'] = df['Incident Date'].fillna(df['Date Received'])
df['Date Received'] = df['Date Received'].fillna(df['Incident Date'])

# Count rows with no 'Incident Date'
missing_incident_date = df['Incident Date'].isnull().sum()
print(f"\nNumber of rows with no 'Incident Date': {missing_incident_date}")

# Count rows with no 'Date Received'
missing_received_date = df['Date Received'].isnull().sum()
print(f"Number of rows with no 'Date Received': {missing_received_date}")

both_missing = df[df['Incident Date'].isna() & df['Date Received'].isna()]
print(f"Rows where both 'Incident Date' and 'Date Received' are missing: {len(both_missing)}")

Replacing empty Incident Dates with Date Received

Number of rows with no 'Incident Date': 124
Number of rows with no 'Date Received': 124
Rows where both 'Incident Date' and 'Date Received' are missing: 124


### Drop rows where both dates missing

In [8]:
df = df[~(df['Incident Date'].isna() & df['Date Received'].isna())] # Drop n/a dates

### Convert missing airline names to 'n/a' and show 

In [9]:
# Handle missing 'Airline Name' as 'N/A'
df['Airline Name'] = df['Airline Name'].fillna('N/A')

In [10]:
# Show all airlines (including 'N/A')
all_airlines = df['Airline Name'].unique()
print(f"\nTotal # distinct airlines available ({len(all_airlines)} before preprocess):")
print(sorted(all_airlines))

# Number of rows per airline (including 'N/A')
airline_counts = df['Airline Name'].value_counts(dropna=False)
print("\nNumber of rows per airline (including 'N/A'):")
print(airline_counts)
num_airlines = df['Airline Name'].dropna().apply(lambda x: x.strip()).replace('', pd.NA).dropna().nunique()
print(f"\nNumber of distinct airlines available: {num_airlines}")
print(f"\nUnited Airlines? {df['Airline Name'].str.strip().str.lower().isin(['united', 'united airlines']).any()}")



Total # distinct airlines available (319 before preprocess):
['-', 'ATA Aerocondor', 'Aer Lingus', 'Aero California', 'Aero Costa Rica', 'Aero Flot', 'Aero Lineas Mexicanas J S S A De C V', 'Aero Lloyd', 'Aero Mexico', 'Aero Peru', 'AeroGal', 'Aeroflot Russian International', 'Aerolineas Argentinas', 'Aerolitoral', 'Aeromar', 'Aerosvit Airlines', 'Aerosweet Airlines', 'Air 2000 Ltd Great Britain', 'Air Afrique', 'Air Aruba', 'Air Atlanta Icelandic', 'Air Atlantic Ltd', 'Air Belgium International S A', 'Air Berlin', 'Air Botina', 'Air Canada', 'Air China', 'Air Europa', 'Air France', 'Air India', 'Air Inter Europe', 'Air Jamaica', 'Air Jamaica  ', 'Air Labrador', 'Air Malta', 'Air New Zealand', 'Air New Zealand  ', 'Air Nippon', 'Air Pacific', 'Air Pacific  ', 'Air Phillipines', 'Air Portugal', 'Air Portugal  ', 'Air Southwest Ltd', 'Air Tahiti Nui', 'Air Tran Airlines(do not use)', 'Air Ukraine International', 'Air Ukraine International  ', 'AirTran Airlines', 'Aires Airline', 'Alaska

### 

In [11]:
# Convert UAL to "United"
df['Airline Name'] = df['Airline Name'].apply(lambda x: x.strip() if isinstance(x, str) else x)
df.loc[df['Airline Name'] == 'UAL', 'Airline Name'] = 'United'


In [12]:
# Show all airlines (including 'N/A')
all_airlines = df['Airline Name'].unique()
print(f"\nTotal # distinct airlines available ({len(all_airlines)} before preprocess):")
print(sorted(all_airlines))

# Number of rows per airline (including 'N/A')
airline_counts = df['Airline Name'].value_counts(dropna=False)
print("\nNumber of rows per airline (including 'N/A'):")
print(airline_counts)
num_airlines = df['Airline Name'].dropna().apply(lambda x: x.strip()).replace('', pd.NA).dropna().nunique()
print(f"\nNumber of distinct airlines available: {num_airlines}")
print(f"\nUnited Airlines? {df['Airline Name'].str.strip().str.lower().isin(['united', 'united airlines']).any()}")



Total # distinct airlines available (234 before preprocess):
['-', 'ATA Aerocondor', 'Aer Lingus', 'Aero California', 'Aero Costa Rica', 'Aero Flot', 'Aero Lineas Mexicanas J S S A De C V', 'Aero Lloyd', 'Aero Mexico', 'Aero Peru', 'AeroGal', 'Aeroflot Russian International', 'Aerolineas Argentinas', 'Aerolitoral', 'Aeromar', 'Aerosvit Airlines', 'Aerosweet Airlines', 'Air 2000 Ltd Great Britain', 'Air Afrique', 'Air Aruba', 'Air Atlanta Icelandic', 'Air Atlantic Ltd', 'Air Belgium International S A', 'Air Berlin', 'Air Botina', 'Air Canada', 'Air China', 'Air Europa', 'Air France', 'Air India', 'Air Inter Europe', 'Air Jamaica', 'Air Labrador', 'Air Malta', 'Air New Zealand', 'Air Nippon', 'Air Pacific', 'Air Phillipines', 'Air Portugal', 'Air Southwest Ltd', 'Air Tahiti Nui', 'Air Tran Airlines(do not use)', 'Air Ukraine International', 'AirTran Airlines', 'Aires Airline', 'Alaska Airlines', 'Alitalia', 'All Nippon Airways', 'Allegheny Airlines Inc', 'Allegiant Air', 'Aloha Airlines

### Check median and average for claim amount and close amount for SeaTac, between 4/29/2012 - 12/31/2015

In [16]:
# Filter by date range and airport code "SEA"
filtered_df = df[
    (df['Incident Date'] >= '2012-04-29') &
    (df['Incident Date'] <= '2015-12-31') &
    (df['Airport Code'] == 'SEA')
]

# Calculate median and average
summary = {
    'Claim Amount': {
        'Median': filtered_df['Claim Amount'].median(),
        'Average': filtered_df['Claim Amount'].mean()
    },
    'Close Amount': {
        'Median': filtered_df['Close Amount'].median(),
        'Average': filtered_df['Close Amount'].mean()
    }
}

# Print the results
for col, stats in summary.items():
    print(f"\n{col} at SEA Airport (2012-04-29 to 2015-12-31):")
    print(f"  Median: ${stats['Median']:.2f}")
    print(f"  Average: ${stats['Average']:.2f}")


Claim Amount at SEA Airport (2012-04-29 to 2015-12-31):
  Median: $nan
  Average: $nan

Close Amount at SEA Airport (2012-04-29 to 2015-12-31):
  Median: $nan
  Average: $nan


C:\Users\vikra\dev\School\dv5320\Final Proj\tsaclaims\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\vikra\dev\School\dv5320\Final Proj\tsaclaims\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


### Save to CSV

In [14]:
df.to_csv("tsa_claims_clean.csv", index=False)